In [1]:
import torch
from transformers import AlbertTokenizer

In [3]:
model = torch.load('saved_model')
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2', do_lower_case=True)
label_dict = {0: 'entailment',1:'contradiction', 2:'neutral'}

In [4]:
def get_label(premise,hypothesis):
    premise_id = tokenizer.encode(premise, add_special_tokens = False)
    hypothesis_id = tokenizer.encode(hypothesis, add_special_tokens = False)
    pair_token_ids = [tokenizer.cls_token_id] + premise_id + [tokenizer.sep_token_id] + hypothesis_id + [tokenizer.sep_token_id]
    premise_len = len(premise_id)
    hypothesis_len = len(hypothesis_id)

    segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
    attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values



    pair_token_ids = torch.tensor(pair_token_ids)
    pair_token_ids = torch.reshape(pair_token_ids,(1,-1))
    seg_ids = torch.reshape(segment_ids,(1,-1))
    mask_ids = torch.reshape(attention_mask_ids,(1,-1))
    labels = torch.tensor([0])

    out = model(pair_token_ids,token_type_ids=seg_ids, 
                                 attention_mask=mask_ids, 
                                 labels=labels)


    softmax = torch.nn.Softmax(dim=0)
    logits = softmax(out.logits)
    preds = torch.argmax(logits,dim=1).numpy()
    label = label_dict[preds[0]]
    print(premise)
    print(hypothesis)
    print(f'Given statemnets are :{label}')

In [5]:
premise = 'A person on a horse jumps over a broken down airplane.'
hypothesis = 'A person is training his horse for a competition.'



In [6]:
get_label(premise,hypothesis)

A person on a horse jumps over a broken down airplane.
A person is training his horse for a competition.
Given statemnets are :neutral
